# CRIADO APENAS PARA TESTES

In [2]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
import google.generativeai as genai
from dotenv import load_dotenv

In [ ]:
# Lendo dados do arquivo parquet
df_deputados = pd.read_parquet('./data/deputados.parquet')

url_expenses = 'https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/despesas'
expenses = []  # Garantir que seja uma lista

for id in df_deputados['id']:
    url_complete = url_expenses.format(id=id)
    answer = requests.get(url_complete, params={"ano": 2024, "mes": 8})
    expenses_json = answer.json()
    
    # Transformar a resposta em DataFrame e adicionar à lista
    df_expenses = pd.DataFrame(expenses_json['dados'])
    if not df_expenses.empty:  # Verificar se o DataFrame não está vazio
        df_expenses['id'] = id
        expenses.append(df_expenses)

    # Link para próxima página
    df_links = pd.DataFrame(expenses_json['links']).set_index('rel').href

    while 'next' in df_links.index:
        answer = requests.get(df_links['next'])
        expenses_json = answer.json()
        df_expenses = pd.DataFrame(expenses_json['dados'])
        
        if not df_expenses.empty:  # Verificar se o DataFrame não está vazio
            df_expenses['id'] = id
            expenses.append(df_expenses)

        # Atualizar os links para a próxima página
        df_links = pd.DataFrame(expenses_json['links']).set_index('rel').href

# Concatenar os DataFrames na lista
df_expenses = pd.concat(expenses, ignore_index=True)
df_expenses = df_expenses[['id', 'tipoDespesa','dataDocumento', 'valorLiquido']]

# Merge para trazer as informações de sigla e afins do deputado
df_expenses = df_expenses.merge(df_deputados, on='id')

# Agrupar por nome, tipo de despesa e data do documento
df_expenses = df_expenses.groupby(['nome', 'tipoDespesa','dataDocumento']).sum()

#Salvando em parquet
df_expenses.to_parquet('./data/serie_despesas_diarias_deputados.parquet')

# Exibir o resultado final
#print(df_expenses.head())


In [6]:
df_expenses.groupby(['nome', 'tipoDespesa','dataDocumento']).sum()

id  \
nome           tipoDespesa                                        dataDocumento                 
AJ Albuquerque COMBUSTÍVEIS E LUBRIFICANTES.                      2024-08-01T00:00:00  409098   
                                                                  2024-08-02T00:00:00  613647   
                                                                  2024-08-03T00:00:00  818196   
                                                                  2024-08-04T00:00:00  409098   
                                                                  2024-08-07T00:00:00  204549   
...                                                                                       ...   
Átila Lira     PASSAGEM AÉREA - SIGEPA                            2024-08-12T12:00:00  246172   
                                                                  2024-08-23T12:00:00  123086   
               SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPEC... 2024-10-02T00:00:00  123086   
               TELEFONIA                                          2024-08-19T00:00:00  246172   
                                                                  2024-08-24T00:00:00  123086   

                                                                                       valorLiquido  \
nome           tipoDespesa                                        dataDocumento                       
AJ Albuquerque COMBUSTÍVEIS E LUBRIFICANTES.                      2024-08-01T00:00:00        596.79   
                                                                  2024-08-02T00:00:00        714.90   
                                                                  2024-08-03T00:00:00        985.26   
                                                                  2024-08-04T00:00:00        560.01   
                                                                  2024-08-07T00:00:00        260.03   
...                                                                                             ...   
Átila Lira     PASSAGEM AÉREA - SIGEPA                            2024-08-12T12:00:00       1568.51   
                                                                  2024-08-23T12:00:00      -1824.54   
               SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPEC... 2024-10-02T00:00:00       8499.96   
               TELEFONIA                                          2024-08-19T00:00:00        314.98   
                                                                  2024-08-24T00:00:00        116.19   

                                                                                                                                     uri  \
nome           tipoDespesa                                        dataDocumento                                                            
AJ Albuquerque COMBUSTÍVEIS E LUBRIFICANTES.                      2024-08-01T00:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
                                                                  2024-08-02T00:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
                                                                  2024-08-03T00:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
                                                                  2024-08-04T00:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
                                                                  2024-08-07T00:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
...                                                                                                                                  ...   
Átila Lira     PASSAGEM AÉREA - SIGEPA                            2024-08-12T12:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
                                                                  2024-08-23T12:00:00  https://dadosabertos.camara.leg.br/api/v2/depu...   
               SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPEC... 2024-10-02T00:00:00  ht

In [10]:
df_expenses.groupby(['tipoDespesa','dataDocumento','nome']).sum()

ano  \
tipoDespesa                                 dataDocumento       nome                        
AQUISIÇÃO DE TOKENS E CERTIFICADOS DIGITAIS 2024-08-19T00:00:00 Carol Dartora        2024   
ASSINATURA DE PUBLICAÇÕES                   2024-07-19T00:00:00 Fernanda Melchionna  2024   
                                            2024-07-23T00:00:00 Carlos Zarattini     2024   
                                                                Fernanda Melchionna  2024   
                                                                Heitor Schuch        2024   
...                                                                                   ...   
TELEFONIA                                   2024-09-10T00:00:00 Tadeu Veneri         2024   
                                                                Tiririca             2024   
                                            2024-09-12T00:00:00 Domingos Sávio       2024   
                                            2024-09-14T00:00:00 Fernando Monteiro    2024   
                                            2024-10-01T00:00:00 Gabriel Nunes        2024   

                                                                                     mes  \
tipoDespesa                                 dataDocumento       nome                       
AQUISIÇÃO DE TOKENS E CERTIFICADOS DIGITAIS 2024-08-19T00:00:00 Carol Dartora          8   
ASSINATURA DE PUBLICAÇÕES                   2024-07-19T00:00:00 Fernanda Melchionna    8   
                                            2024-07-23T00:00:00 Carlos Zarattini       8   
                                                                Fernanda Melchionna    8   
                                                                Heitor Schuch          8   
...                                                                                  ...   
TELEFONIA                                   2024-09-10T00:00:00 Tadeu Veneri           8   
                                                                Tiririca               8   
                                            2024-09-12T00:00:00 Domingos Sávio         8   
                                            2024-09-14T00:00:00 Fernando Monteiro      8   
                                            2024-10-01T00:00:00 Gabriel Nunes          8   

                                                                                     codDocumento  \
tipoDespesa                                 dataDocumento       nome                                
AQUISIÇÃO DE TOKENS E CERTIFICADOS DIGITAIS 2024-08-19T00:00:00 Carol Dartora             7789801   
ASSINATURA DE PUBLICAÇÕES                   2024-07-19T00:00:00 Fernanda Melchionna       7786629   
                                            2024-07-23T00:00:00 Carlos Zarattini          7790138   
                                                                Fernanda Melchionna       7786729   
                                                                Heitor Schuch             7798910   
...                                                                                           ...   
TELEFONIA                                   2024-09-10T00:00:00 Tadeu Veneri              7812522   
                                                                Tiririca                  7804897   
                                            2024-09-12T00:00:00 Domingos Sávio            7815234   
                                            2024-09-14T00:00:00 Fernando Monteiro         7816492   
                                            2024-10-01T00:00:00 Gabriel Nunes             7822766   

                                                                                    tipoDocumento  \
tipoDespesa                                 dataDocumento       nome                                
AQUISIÇÃO DE TOKENS E CERTIFICADOS DIGITAIS 2024-08-19T00:00:00 Carol Dartora         Nota Fiscal   
ASSINATURA DE PUBLICAÇÕES                   2024-07-19T00:00:0

In [1]:
import pandas as pd

In [52]:
df_insi = pd.read_parquet('./data/serie_despesas_diarias_deputados.parquet')

In [54]:
df_insi.columns

Index(['nome', 'tipoDespesa', 'dataDocumento', 'id', 'valorLiquido', 'uri',
       'siglaPartido', 'uriPartido', 'siglaUf', 'idLegislatura', 'urlFoto',
       'email'],
      dtype='object')

In [23]:
# Carregando os dados
df = pd.read_parquet('./data/serie_despesas_diarias_deputados.parquet')

In [28]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Carregando os dados
df = pd.read_parquet('./data/serie_despesas_diarias_deputados.parquet')

# Convertendo a coluna 'dataDocumento' para datetime
df['dataDocumento'] = pd.to_datetime(df['dataDocumento'])

# Análise 1: Despesas totais por deputado
despesas_por_deputado = df.groupby('nome')['valorLiquido'].sum().reset_index()
fig1 = px.bar(despesas_por_deputado, x='nome', y='valorLiquido',
              title='Despesas Totais por Deputado',
              labels={'nome': 'Nome do Deputado', 'valorLiquido': 'Valor Total (R$)'},
              height=600)
fig1.update_layout(xaxis={'categoryorder':'total descending'}) #ordena pelo maior valor


# Análise 2: Distribuição das despesas por tipo
despesas_por_tipo = df.groupby('tipoDespesa')['valorLiquido'].sum().reset_index()
fig2 = px.pie(despesas_por_tipo, values='valorLiquido', names='tipoDespesa',
              title='Distribuição das Despesas por Tipo',
              height=600)


# Análise 3: Despesas ao longo do tempo
despesas_ao_longo_do_tempo = df.groupby('dataDocumento')['valorLiquido'].sum().reset_index()
fig3 = px.line(despesas_ao_longo_do_tempo, x='dataDocumento', y='valorLiquido',
              title='Despesas ao Longo do Tempo',
              labels={'dataDocumento': 'Data', 'valorLiquido': 'Valor Total (R$)'},
              height=600)


fig1.show()
fig2.show()
fig3.show()

In [15]:
df.columns

Index(['id', 'valorLiquido', 'uri', 'siglaPartido', 'uriPartido', 'siglaUf',
       'idLegislatura', 'urlFoto', 'email'],
      dtype='object')

In [30]:
import requests

In [31]:
url_base = 'https://dadosabertos.camara.leg.br/api/v2/'
url_complete =  url_base + 'preposicoes'
answer = requests.get(url_complete, params={"datainicio": '2024-08-01',
                                            "datafim": '2024-08-31',
                                            "codTema": [40, 46, 62]})

In [50]:
import json

url_base = 'https://dadosabertos.camara.leg.br/api/v2/'
url_complete =  url_base + 'proposicoes'
codTema = [40, 46, 62]
df_proposicoes = None

for cod in codTema:
    answer = requests.get(url_complete, params={"dataInicio": '2024-08-01',
                                                "dataFim": '2024-08-31',
                                                "codTema": cod,
                                                "itens": 10})
    
    if df_proposicoes is None:
        df_proposicoes = pd.DataFrame().from_dict(json.loads(answer.text)['dados'])
    else:
        df_proposicoes = pd.concat([df_proposicoes, pd.DataFrame().from_dict(json.loads(answer.text)['dados'])])


In [51]:
df_proposicoes

,id,uri,siglaTipo,codTipo,numero,ano,ementa
0,253500,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2597,2024,Estabelece normas gerais em contratos de segur...
1,379298,https://dadosabertos.camara.leg.br/api/v2/prop...,PLP,140,249,2007,"Altera a Lei Complementar nº 101, de 4 de maio..."
2,599485,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6692,2013,"Altera o art. 47 da Lei nº 5.764, de 16 de dez..."
3,955550,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,532,2015,"Modifica os arts. 7º, 9º, 16 e 20 da Lei nº 7...."
4,1786192,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3127,2015,Dispõe sobre aumento do repasse oriundo das re...
5,2074942,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3987,2015,Destina 1% da arrecadação das Loterias Federai...
6,2124303,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6987,2017,Destina a renda líquida de um concurso da lote...
7,2130732,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7474,2017,"Acrescenta art. 3º-A à Lei nº 10.214, de 27 de..."
8,2141516,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7877,2017,Dá nova redação a dispositivos da Lei nº 9.613...
9,2164554,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,9255,2017,Acrescenta novo § 4º ao art. 52 da Lei nº 8.07...


In [55]:
prop = pd.read_parquet('./data/proposicoes.parquet')

In [56]:
prop

,id,uri,siglaTipo,codTipo,numero,ano,ementa
0,253500,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2597,2024,Estabelece normas gerais em contratos de segur...
1,379298,https://dadosabertos.camara.leg.br/api/v2/prop...,PLP,140,249,2007,"Altera a Lei Complementar nº 101, de 4 de maio..."
2,599485,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6692,2013,"Altera o art. 47 da Lei nº 5.764, de 16 de dez..."
3,955550,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,532,2015,"Modifica os arts. 7º, 9º, 16 e 20 da Lei nº 7...."
4,1786192,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3127,2015,Dispõe sobre aumento do repasse oriundo das re...
5,2074942,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,3987,2015,Destina 1% da arrecadação das Loterias Federai...
6,2124303,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,6987,2017,Destina a renda líquida de um concurso da lote...
7,2130732,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7474,2017,"Acrescenta art. 3º-A à Lei nº 10.214, de 27 de..."
8,2141516,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,7877,2017,Dá nova redação a dispositivos da Lei nº 9.613...
9,2164554,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,9255,2017,Acrescenta novo § 4º ao art. 52 da Lei nº 8.07...
